In [3]:
import torch
import torch.nn as nn
import resnet
import model

maps = 32
dim_feature = 7 * 7
basenet = resnet.resnet18(True, maps=maps)
# lstm = nn.LSTM(dim_feature, dim_feature, 2, bidirectional=True, batch_first=True)
lstm = nn.LSTM(maps * dim_feature, maps * dim_feature, 3, bidirectional=False, batch_first=True)
linear_after_lstm = nn.Linear(maps * dim_feature, maps * dim_feature)

input = torch.randn(4,3,224,224)
feature = basenet(input)
print(feature.size())

net = model.Model()
# gaze = net(input)
# print(gaze)


torch.Size([4, 32, 7, 7])


In [4]:
batch_size = feature.size(0)
feature = feature.flatten(1)

feature = linear_after_lstm(feature)

print(batch_size)
print(feature.size())

4
torch.Size([4, 1568])


In [5]:
feature = torch.randn(10,4,1568)
lstm_feature, _ = lstm(feature)
print(len(_))
# print(_)
print(lstm_feature.size())

# print(lstm_feature[-1, :])
# lstm_feature = lstm_feature[-1, :]

2
torch.Size([10, 4, 1568])


In [6]:
d_model = maps * dim_feature
nhead = 8
dropout = 0.1
self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout,batch_first=True)

q=k=lstm_feature
src2, weight = self_attn(q,k,lstm_feature)
print(src2.size())
print(weight.size())
# print(src2)
# print(weight)
# print(weight[0])


torch.Size([10, 4, 1568])
torch.Size([10, 4, 4])


In [7]:
cls_token = nn.Parameter(torch.randn(1, 1, 32))
cls = cls_token.repeat((1, 3, 2))
print(cls.size())
print(cls)



# tr_feature = feature.permute(2, 0, 1)

# print(tr_feature.size())

torch.Size([1, 3, 64])
tensor([[[ 0.0127,  0.2547, -0.5900, -0.5150,  0.8341,  0.3086, -1.2778,
           1.0204, -0.1114,  0.6874, -0.4209,  0.2231,  0.0132, -0.7971,
          -1.0226,  0.4232, -0.7172, -0.0988,  1.1001, -0.0039,  0.2722,
           0.6263, -0.5229, -0.9726,  0.1074, -1.3913,  0.1393, -1.0538,
          -0.7352, -0.3824,  0.6050, -0.7491,  0.0127,  0.2547, -0.5900,
          -0.5150,  0.8341,  0.3086, -1.2778,  1.0204, -0.1114,  0.6874,
          -0.4209,  0.2231,  0.0132, -0.7971, -1.0226,  0.4232, -0.7172,
          -0.0988,  1.1001, -0.0039,  0.2722,  0.6263, -0.5229, -0.9726,
           0.1074, -1.3913,  0.1393, -1.0538, -0.7352, -0.3824,  0.6050,
          -0.7491],
         [ 0.0127,  0.2547, -0.5900, -0.5150,  0.8341,  0.3086, -1.2778,
           1.0204, -0.1114,  0.6874, -0.4209,  0.2231,  0.0132, -0.7971,
          -1.0226,  0.4232, -0.7172, -0.0988,  1.1001, -0.0039,  0.2722,
           0.6263, -0.5229, -0.9726,  0.1074, -1.3913,  0.1393, -1.0538,
        

In [8]:
x = nn.Parameter(torch.randn(2,3))
print(x)
print(torch.cat((x,x,x),0))
print(torch.cat((x,x,x),1))

Parameter containing:
tensor([[-0.8466,  0.3534, -0.7178],
        [ 0.0724, -0.3986,  1.7843]], requires_grad=True)
tensor([[-0.8466,  0.3534, -0.7178],
        [ 0.0724, -0.3986,  1.7843],
        [-0.8466,  0.3534, -0.7178],
        [ 0.0724, -0.3986,  1.7843],
        [-0.8466,  0.3534, -0.7178],
        [ 0.0724, -0.3986,  1.7843]], grad_fn=<CatBackward0>)
tensor([[-0.8466,  0.3534, -0.7178, -0.8466,  0.3534, -0.7178, -0.8466,  0.3534,
         -0.7178],
        [ 0.0724, -0.3986,  1.7843,  0.0724, -0.3986,  1.7843,  0.0724, -0.3986,
          1.7843]], grad_fn=<CatBackward0>)


In [9]:
encoder_layer = model.TransformerEncoderLayer(maps*4, nhead, 512, dropout)

encoder_norm = nn.LayerNorm(maps*4)
# num_encoder_layer: deeps of layers

encoder = model.TransformerEncoder(encoder_layer, 6, encoder_norm)

In [10]:
from easydict import EasyDict as edict
import yaml
import ctools
import importlib
import os
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter


config = "./config/config_gaze360.yaml"

person = None

config = edict(yaml.load(open(config), Loader=yaml.FullLoader))

config = config.train

config.person = person

config.device = 1

print("=====================>> (Begin) Training params << =======================")

print(ctools.DictDumps(config))

print("=====================>> (End) Training params << =======================")

dataloader = importlib.import_module("reader." + config.reader)
device = torch.device(config.device)
torch.cuda.set_device(device)
cudnn.benchmark = True

data = config.data
save = config.save
params = config.params
average_loss_num = config.ave_loss

# writer = SummaryWriter()

print("========================> Read Data <========================")
if config.person is not None:
    if data.isFolder:
        data, folder = ctools.readfolder(data, [config.person], reverse=True)

    save_name = folder[config.person]

    dataset = dataloader.loader(data, params.batch_size, shuffle=True, num_workers=6)

    save_path = os.path.join(save.metapath, save.folder, f"checkpoint/{save_name}")

else:
    if data.isFolder:
        data, _ = ctools.readfolder(data)

    dataset = dataloader.loader(data, params.batch_size, shuffle=False, num_workers=8)

    save_path = os.path.join(save.metapath, save.folder, "checkpoint")

if not os.path.exists(save_path):
    os.makedirs(save_path)
    

=====================>> (Begin) Training params << =======================
{
    "params": {
        "batch_size": 100,
        "epoch": 100,
        "lr": 0.0001,
        "decay": 0.5,
        "decay_step": 5,
        "warmup": 0
    },
    "save": {
        "metapath": "/home/zjx/Gaze/Output/HybirdGaze",
        "folder": "gaze360_10000",
        "model_name": "trans6",
        "step": 5
    },
    "data": {
        "image": "/home/zjx/Gaze/Datasets/gaze360_processed_1/Image",
        "label": "/home/zjx/Gaze/Datasets/gaze360_processed_1/Label/train.label",
        "header": true,
        "name": "gaze360",
        "isFolder": false
    },
    "pretrain": {
        "enable": false,
        "path": "PLACEHOLDER",
        "device": "PLACEHOLDER"
    },
    "device": 1,
    "ave_loss": 1000,
    "reader": "reader",
    "person": null
}
=====================>> (End) Training params << =======================
========================> Read Data <========================
-- [Read Data]: So

In [11]:
dataset

In [12]:
for data, label in dataset:
    break

print(data.face.size())
print(len(data.name))
for name in data.name:
    print(name)
print(label.size())
print(label)

torch.Size([100, 4, 3, 224, 224])
100
rec_000/head/000015/000003.jpg
rec_000/head/000015/000004.jpg
rec_000/head/000015/000005.jpg
rec_000/head/000015/000006.jpg
rec_000/head/000015/000007.jpg
rec_000/head/000015/000008.jpg
rec_000/head/000015/000009.jpg
rec_000/head/000052/000009.jpg
rec_000/head/000015/000010.jpg
rec_000/head/000015/000011.jpg
rec_000/head/000015/000012.jpg
rec_000/head/000015/000013.jpg
rec_000/head/000015/000014.jpg
rec_000/head/000015/000015.jpg
rec_000/head/000015/000016.jpg
rec_000/head/000015/000017.jpg
rec_000/head/000015/000018.jpg
rec_000/head/000015/000019.jpg
rec_000/head/000015/000020.jpg
rec_000/head/000015/000021.jpg
rec_000/head/000015/000022.jpg
rec_000/head/000015/000023.jpg
rec_000/head/000015/000024.jpg
rec_000/head/000015/000025.jpg
rec_000/head/000015/000026.jpg
rec_000/head/000015/000027.jpg
rec_000/head/000015/000028.jpg
rec_000/head/000015/000029.jpg
rec_000/head/000015/000030.jpg
rec_000/head/000015/000031.jpg
rec_000/head/000015/000032.jpg
r

In [13]:
for i, (data,label) in enumerate(dataset):
    print(data.face.size())
#     print(data)
#     print(label)
    break

torch.Size([100, 4, 3, 224, 224])


In [14]:
import model as model

net = model.Model()
net.train()
out = net(data['face'])
out.size()

torch.Size([100, 2])

In [15]:
from resnet import resnet18

input = data['face']
print(input.size())
batch_size = input.size(0)
input = torch.flatten(input, start_dim=0, end_dim=1)
print(input.size())
base_model = resnet18(True, maps=maps)
feature = base_model(input)
print(feature.size())
# print(feature.size())
feature = feature.flatten(1)
print(feature.size())
linear1 = nn.Linear(maps * dim_feature, maps * dim_feature)
feature = linear1(feature)
print(feature.size())

feature = torch.reshape(feature, (batch_size, feature.size(0)//batch_size, feature.size(1)))
print(feature.size())

torch.Size([100, 4, 3, 224, 224])
torch.Size([400, 3, 224, 224])
torch.Size([400, 32, 7, 7])
torch.Size([400, 1568])
torch.Size([400, 1568])
torch.Size([100, 4, 1568])


In [ ]:
import scipy.io as sio

root = "/home/zjx/Gaze/Datasets/gaze360"
msg = sio.loadmat(os.path.join(root, "metadata.mat"))

msg


In [ ]:
import reader.reader as dataloader

import ctools

if data.isFolder:
    data, _ = ctools.readfolder(data)

dataset = dataloader.loader(data, params.batch_size, shuffle=True, num_workers=8)

In [ ]:
a = []
isinstance(a,list)

In [ ]:
with open("/home/zjx/Gaze/Datasets/gaze360_processed_1/Label/train.label") as f:
    dataline = f.readlines()

print(len(dataline))
dataline.pop(0)
print(len(dataline))

In [ ]:
line = dataline[4]
print(line)
line = line.strip().split(" ")
print(line)

In [ ]:
items = [1,2,3]
for item in items:
    item+=item
items

In [ ]:
from torchvision import transforms
import cv2
import numpy as np

transforms_ = transforms.Compose([transforms.ToTensor()])

img = cv2.imread("/home/zjx/Gaze/Datasets/gaze360_processed_1/Image/test/Face/1004.jpg")
print(img.shape)
img = transforms_(img)
print(img.shape)

In [ ]:
img = img.unsqueeze(0)
img.size()

In [ ]:
imgs = torch.tensor()
imgs.append(img)
imgs.append(img)
imgs = torch.tensor(imgs)


In [ ]:
import torch

a = torch.arange(32)
b = torch.reshape(a,(2,2,2,2,2))
print(b[1,1])
# c = torch.reshape(b,(4,2,2,2))
c = torch.flatten(b, start_dim=0, end_dim=1)
print(c.size())
print(c[3])

d = torch.reshape(c,(2,2,2,2,2))
print(d[1,1])


In [ ]:
b.dim()
b.size(0)

In [ ]:
a = torch.randn(2,4,4)
print(a)
a = a[:,-1,:].unsqueeze(1)
print(a.size())
print(a)
a = a.unsqueeze(1)
print(a.size())
print(a)

In [18]:
out = torch.randn(3,2)
label = torch.randn(3,2)
print(out)
print(label)

tensor([[ 0.0783, -1.1302],
        [ 0.6067,  0.0837],
        [ 0.6113,  0.3518]])
tensor([[-0.5985,  1.3614],
        [ 0.2129,  0.2008],
        [-0.8670, -0.1806]])


In [24]:
import torch
from torch import nn
loss_func = nn.MSELoss(reduction='mean')

loss = loss_func(out,label)
loss

tensor(1.5506)